<a href="https://colab.research.google.com/github/EoghanPower/pr/blob/main/Create_a_Tweets_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#Creating a Data Frame of Tweets on S&P 500 Companies

In [ ]:
# import libraries
import tweepy, json 
from textblob import TextBlob 
from wordcloud import wordcloud
import pandas as pd
import numpy as np 
import re
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import webbrowser
import time 
import requests
import requests_oauthlib
import os

In [ ]:
#authentication store 0auth credentials in variables
consumer_key = 'eHrpqTZVq5bfGdjC8juN7fSCs'
consumer_secret = 'itfojeERaMFXtwQFI1emQNT3vwCHs414OJdOXlpN2RaBAw3MIz'
AccessToken = '264845011-WhC9gKJlC3TayNsXg4ViBIuwaK0D5Sof3lxPTZeN'
AccessTokenSecret = 'dE5difJJmYP5UYllqeQwTsovcoXyvco9Ww2QMxiR365D3'

In [ ]:
#Pass 0auth details to tweepy's 0auth handlet 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(AccessToken, AccessTokenSecret)

In [ ]:
##SetAPIobject
api = tweepy.API(auth, wait_on_rate_limit= True)

In [ ]:
#StreamListener
class MyStreamListener(tweepy.StreamListener):
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.file = open("tweets.txt", "w")

    def on_status(self, status):
        tweet = status._json
        self.file.write( json.dumps(tweet) + '\n' )
        self.num_tweets += 1
        if self.num_tweets < 10001:
            return True
        else:
            return False
        self.file.close()

    def on_error(self, status):
        print(status)

In [ ]:
#initialize stream listener
l = MyStreamListener()
#Create stream object with authentication
stream = tweepy.Stream(auth, l)
#Filter Twitter Streams to capture data by the keywords:
stream.filter(languages = ['en'],track=['Apple', 'Google', 'Microsoft','Amazon',  'Facebook', 'JPMorgan',   'Johnson & Johnson', '  Exxon', 'Bank of America', 'Wal-Mart, Wells Fargo', 'Berkshire Hathaway', 'AT&T ', 'Tesla', 'Gamestop' ])

In [ ]:
#import package
import json

#String of path to file: tweets_data_path
tweets_data_path = 'tweets.txt'

#initialize empty list to store tweets: tweets_data
tweets_data = []

In [ ]:
#open connection to file
tweets_file = open(tweets_data_path, 'r')

In [ ]:
# Read in tweets and store in list: tweets_data
for line in tweets_file:
  tweet = json.loads(line)
  tweets_data.append(tweet)

In [ ]:
#close connection to file
tweets_file.close()


In [ ]:
print(tweets_data[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms'])


In [ ]:
# Build DataFrame of tweet texts and languages
df = pd.DataFrame(tweets_data, columns= ['text','created_at'])

In [ ]:
print(df.head(10))

                                                text                      created_at
0  @snsnkeun5 @SB19Official Ito po :)\n\nhttps://...  Tue Dec 21 12:35:07 +0000 2021
1  RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...  Tue Dec 21 12:35:07 +0000 2021
2  RT @TopGoal_NFT: After completing all the step...  Tue Dec 21 12:35:07 +0000 2021
3  RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...  Tue Dec 21 12:35:07 +0000 2021
4  Where political dysfunction co-exists with a b...  Tue Dec 21 12:35:07 +0000 2021
5  RT @TejuKaDevar: Nothing just two pandas eatin...  Tue Dec 21 12:35:07 +0000 2021
6  RT @caughttin4k: amazon not even trying anymor...  Tue Dec 21 12:35:07 +0000 2021
7  RT @GOAT_Official_: @VootSelect Karan has not ...  Tue Dec 21 12:35:07 +0000 2021
8  RT @AdoptionsUk: Please retweet to help find t...  Tue Dec 21 12:35:07 +0000 2021
9                        RT @cjp_still: Read it all.  Tue Dec 21 12:35:07 +0000 2021


In [ ]:
#clean the tweets
def cleanTwt(twt): 
  twt = re.sub('#apple','Apple', twt)
  twt = re.sub('#google','Google', twt) 
  twt = re.sub('#microsoft','Microsoft', twt) 
  twt = re.sub('#amazon','Amazon', twt) 
  twt = re.sub('#facebook','Facebook', twt) 
  twt = re.sub('#JPMorgan','JPMorgan', twt) 
  twt = re.sub('#Johnson&Johnson','Johnson & Johnson', twt) 
  twt = re.sub('#Exxon','Exxon', twt) 
  twt = re.sub('#BankofAmerica','Bank of America', twt) 
  twt = re.sub('#Wal-Mart','Wal-Mart', twt) 
  twt = re.sub('#Wells Fargo','Wells Fargo', twt) 
  twt = re.sub('#BerkshireHathaway','Berkshire Hathaway', twt) 
  twt = re.sub('#Gamestop','Gamestop', twt) 
  twt = re.sub('#Tesla ','Tesla', twt)#removes the # 
  twt =re.sub('#[A-Za-z0-9]+', '',twt) #removes any strings with a # 
  twt = re.sub('\\n', '', twt) #removes the '\n' string
  twt = re.sub('https?:\/\/\S+', '', twt) #removes any hyperlinks
  twt = re.sub('RT', '', twt) #cleans retweets
  twt = re.sub('@[A-Za-z0-9]+', ',', twt)#Removes tagged accounts
  return twt

In [ ]:
#clean them using the above function

In [ ]:
df['Cleaned_Tweets'] = df['text'].apply(cleanTwt)

In [ ]:
#show the set 
df.head(10)

,text,created_at,Cleaned_Tweets
0,@snsnkeun5 @SB19Official Ito po :)\n\nhttps://...,Tue Dec 21 12:35:07 +0000 2021,", , Ito po :) BBHTS BACK ON TOP, …"
1,RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...,Tue Dec 21 12:35:07 +0000 2021,",_man159: #พี่ตูนวิ่งทําไมFacebook : Poetry o..."
2,RT @TopGoal_NFT: After completing all the step...,Tue Dec 21 12:35:07 +0000 2021,",_NFT: After completing all the steps detaile..."
3,RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...,Tue Dec 21 12:35:07 +0000 2021,",_man159: #พี่ตูนวิ่งทําไมFacebook : Poetry o..."
4,Where political dysfunction co-exists with a b...,Tue Dec 21 12:35:07 +0000 2021,Where political dysfunction co-exists with a b...
5,RT @TejuKaDevar: Nothing just two pandas eatin...,Tue Dec 21 12:35:07 +0000 2021,",: Nothing just two pandas eating apple 🍎😭"
6,RT @caughttin4k: amazon not even trying anymor...,Tue Dec 21 12:35:07 +0000 2021,",: amazon not even trying anymore"
7,RT @GOAT_Official_: @VootSelect Karan has not ...,Tue Dec 21 12:35:07 +0000 2021,",_Official_: , Karan has not eaten a proper m..."
8,RT @AdoptionsUk: Please retweet to help find t...,Tue Dec 21 12:35:07 +0000 2021,",: Please retweet to help find the ORIGINAL O..."
9,RT @cjp_still: Read it all.,Tue Dec 21 12:35:07 +0000 2021,",_still: Read it all."


In [ ]:
#polarity not working for non english tweets, will need to filter stream further

In [ ]:
#create a function to get subjectivity
def getSubjectivity(twt):
  return TextBlob(twt).sentiment.subjectivity
#create a function to get polarity
def getPolarity(twt):
  return TextBlob(twt).sentiment.polarity


In [ ]:
#create two new columns called subjectivity and polarity
df['Subjectivity'] = df['Cleaned_Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Cleaned_Tweets'].apply(getPolarity)

In [ ]:
#show data
df.head(20)

,text,created_at,Cleaned_Tweets,Subjectivity,Polarity
0,@snsnkeun5 @SB19Official Ito po :)\n\nhttps://...,Tue Dec 21 12:35:07 +0000 2021,", , Ito po :) BBHTS BACK ON TOP, …",0.500000,0.333333
1,RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...,Tue Dec 21 12:35:07 +0000 2021,",_man159: #พี่ตูนวิ่งทําไมFacebook : Poetry o...",0.000000,0.000000
2,RT @TopGoal_NFT: After completing all the step...,Tue Dec 21 12:35:07 +0000 2021,",_NFT: After completing all the steps detaile...",0.408333,0.200000
3,RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...,Tue Dec 21 12:35:07 +0000 2021,",_man159: #พี่ตูนวิ่งทําไมFacebook : Poetry o...",0.000000,0.000000
4,Where political dysfunction co-exists with a b...,Tue Dec 21 12:35:07 +0000 2021,Where political dysfunction co-exists with a b...,0.580000,-0.200000
5,RT @TejuKaDevar: Nothing just two pandas eatin...,Tue Dec 21 12:35:07 +0000 2021,",: Nothing just two pandas eating apple 🍎😭",0.000000,0.000000
6,RT @caughttin4k: amazon not even trying anymor...,Tue Dec 21 12:35:07 +0000 2021,",: amazon not even trying anymore",0.000000,0.000000
7,RT @GOAT_Official_: @VootSelect Karan has not ...,Tue Dec 21 12:35:07 +0000 2021,",_Official_: , Karan has not eaten a proper m...",0.550000,0.000000
8,RT @AdoptionsUk: Please retweet to help find t...,Tue Dec 21 12:35:07 +0000 2021,",: Please retweet to help find the ORIGINAL O...",0.525000,0.287500
9,RT @cjp_still: Read it all.,Tue Dec 21 12:35:07 +0000 2021,",_still: Read it all.",0.000000,0.000000


In [ ]:
#Create a function to get the sentiment text
def getSentiment(score):
  if score <0:
    return 'Negative'
  elif score == 0:
      return 'Neutral'
  else:
      return 'Positive'

In [ ]:
#Create column to store the text sentiment
df['Sentiment'] = df['Polarity'].apply(getSentiment)
#Show the data
df.head(7)

,text,created_at,Cleaned_Tweets,Subjectivity,Polarity,Sentiment
0,@snsnkeun5 @SB19Official Ito po :)\n\nhttps://...,Tue Dec 21 12:35:07 +0000 2021,", , Ito po :) BBHTS BACK ON TOP, …",0.500000,0.333333,Positive
1,RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...,Tue Dec 21 12:35:07 +0000 2021,",_man159: #พี่ตูนวิ่งทําไมFacebook : Poetry o...",0.000000,0.000000,Neutral
2,RT @TopGoal_NFT: After completing all the step...,Tue Dec 21 12:35:07 +0000 2021,",_NFT: After completing all the steps detaile...",0.408333,0.200000,Positive
3,RT @Old_man159: #พี่ตูนวิ่งทําไม\nFacebook : P...,Tue Dec 21 12:35:07 +0000 2021,",_man159: #พี่ตูนวิ่งทําไมFacebook : Poetry o...",0.000000,0.000000,Neutral
4,Where political dysfunction co-exists with a b...,Tue Dec 21 12:35:07 +0000 2021,Where political dysfunction co-exists with a b...,0.580000,-0.200000,Negative
5,RT @TejuKaDevar: Nothing just two pandas eatin...,Tue Dec 21 12:35:07 +0000 2021,",: Nothing just two pandas eating apple 🍎😭",0.000000,0.000000,Neutral
6,RT @caughttin4k: amazon not even trying anymor...,Tue Dec 21 12:35:07 +0000 2021,",: amazon not even trying anymore",0.000000,0.000000,Neutral
